<a href="https://colab.research.google.com/github/nisargpatel28/cuDF_cuML_cuGraph/blob/main/RAPIDS_cuDF's_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

cuDF is a Python GPU DataFrame library (built on the Apache Arrow columnar memory format) for loading, joining, aggregating, filtering, and otherwise manipulating tabular data using a DataFrame style API in the style of pandas. cuDF now provides a pandas accelerator mode (`cudf.pandas`), allowing you to bring accelerated computing to your pandas workflows without requiring any code change.

In [ ]:
import cudf

In [ ]:
!pip install plotly-express

We'll also install plotly-express for visualizing data.

In [ ]:
!pip install plotly-express

In [ ]:
!pip install plotly-express

Download the data
The data we'll be working with is the Parking Violations Issued - Fiscal Year 2022 dataset from NYC Open Data. We're downloading a copy of this dataset from an s3 bucket hosted by NVIDIA to provide faster download speeds. We'll start by downloading the data. This should take about 30 seconds.

<a name='s1-2.1'></a>
### Data License and Terms ###
As this dataset originates from the NYC Open Data Portal, it's governed by their license and terms of use.

**Are there restrictions on how I can use Open Data?**
> Open Data belongs to all New Yorkers. There are no restrictions on the use of Open Data. Refer to Terms of Use for more information.

**[Terms of Use](https://opendata.cityofnewyork.us/overview/#termsofuse)**
> By accessing datasets and feeds available through NYC Open Data, the user agrees to all of the Terms of Use of NYC.gov as well as the Privacy Policy for NYC.gov. The user also agrees to any additional terms of use defined by the agencies, bureaus, and offices providing data. Public data sets made available on NYC Open Data are provided for informational purposes. The City does not warranty the completeness, accuracy, content, or fitness for any particular purpose or use of any public data set made available on NYC Open Data, nor are any such warranties to be implied or inferred with respect to the public data sets furnished therein.

> The City is not liable for any deficiencies in the completeness, accuracy, content, or fitness for any particular purpose or use of any public data set, or application utilizing such data set, provided by any third party.

> Submitting City Agencies are the authoritative source of data available on NYC Open Data. These entities are responsible for data quality and retain version control of data sets and feeds accessed on the Site. Data may be updated, corrected, or refreshed at any time.

In [ ]:
!wget https://data.rapids.ai/datasets/nyc_parking/nyc_parking_violations_2022.parquet

--2025-12-04 13:30:11--  https://data.rapids.ai/datasets/nyc_parking/nyc_parking_violations_2022.parquet
Resolving data.rapids.ai (data.rapids.ai)... 3.167.112.123, 3.167.112.60, 3.167.112.70, ...
Connecting to data.rapids.ai (data.rapids.ai)|3.167.112.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 474211285 (452M) [binary/octet-stream]
Saving to: ‘nyc_parking_violations_2022.parquet’

nyc_parking_violati 100%[===================>] 452.24M  45.9MB/s    in 6.6s    

2025-12-04 13:30:18 (68.6 MB/s) - ‘nyc_parking_violations_2022.parquet’ saved [474211285/474211285]



<a name='s1-3'></a>
## Analysis using Standard pandas ##
First, let's use pandas to read in some columns of the dataset:

In [ ]:
import pandas as pd

In [ ]:
# read 5 columns data:
df = pd.read_parquet(
    "nyc_parking_violations_2022.parquet",
    columns=["Registration State", "Violation Description", "Vehicle Body Type", "Issue Date", "Summons Number"]
)

# view a random sample of 10 rows:
df.sample(10)

,Registration State,Violation Description,Vehicle Body Type,Issue Date,Summons Number
10166798,NY,FAILURE TO STOP AT RED LIGHT,SUBN,02/11/2022,5119368384
3966815,CA,40-Fire Hydrant,4DSD,09/08/2021,8981737903
4360306,NY,21-No Parking (street clean),SUBN,10/07/2021,8963923824
15387717,NJ,19-No Stand (bus stop),DELV,06/24/2022,8957649062
2810983,NY,38-Failure to Dsplay Meter Rec,4DSD,08/25/2021,8992731050
3929602,NJ,21-No Parking (street clean),4DSD,09/27/2021,8966805541
4291612,PA,13-No Stand (taxi stand),4DSD,10/22/2021,8901225440
10536220,NY,PHTO SCHOOL ZN SPEED VIOLATION,SUBN,02/28/2022,4770349208
14424174,NY,68-Not Pkg. Comp. w Psted Sign,4DSD,06/08/2022,8987315411
4838737,NY,74-Missing Display Plate,4DSD,10/23/2021,8837629450


**Which parking violation is most commonly committed by vehicles from various U.S states?**
Each record in our dataset contains the state of registration of the offending vehicle, and the type of parking offence. Let's say we want to get the most common type of offence for vehicles registered in different states. We can do this in pandas using a combination of value_counts and GroupBy.head:

Next, we'll try to answer a few questions using the data.

<a name='s1-3.1'></a>
### Which parking violation is most commonly committed by vehicles from various U.S states? ###
Each record in our dataset contains the state of registration of the offending vehicle, and the type of parking offence. Let's say we want to get the most common type of offence for vehicles registered in different states. We can do this in pandas using a combination of [value_counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) and [GroupBy.head](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.head.html):

In [ ]:
(df[["Registration State", "Violation Description"]]  # get only these two columns
 .value_counts()  # get the count of offences per state and per type of offence
 .groupby("Registration State")  # group by state
 .head(1)  # get the first row in each group (the type of offence with the largest count)
 .sort_index()  # sort by state name
 .reset_index()
)

,Registration State,Violation Description,count
0,99,<NA>,17550
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
62,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
63,WA,21-No Parking (street clean),3732
64,WI,14-No Standing,1639
65,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


The code above uses [method chaining](https://tomaugspurger.net/posts/method-chaining/) to combine a series of operations into a single statement. You might find it useful to break the code up into multiple statements and inspect each of the intermediate results!

<a name='s1-3.2'></a>
### Which vehicle body types are most frequently involved in parking violations? ###
We can also investigate which vehicle body types most commonly appear in parking violations.

In [ ]:
(df
 .groupby(["Vehicle Body Type"])
 .agg({"Summons Number": "count"})
 .rename(columns={"Summons Number": "Count"})
 .sort_values(["Count"], ascending=False)
)

,Count
Vehicle Body Type,
SUBN,6449007
4DSD,4402991
VAN,1317899
DELV,436430
PICK,429798
...,...
OLLL,1
OLLE,1
OLHE,1


<a name='s1-3.3'></a>
### How do parking violations vary across days of the week? ###

In [ ]:
weekday_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["Issue Date"] = df["Issue Date"].astype("datetime64[ms]")
df["issue_weekday"] = df["Issue Date"].dt.weekday.map(weekday_names)

df.groupby(["issue_weekday"])["Summons Number"].count().sort_values()

,Summons Number
issue_weekday,
Sunday,462992
Saturday,1108385
Monday,2488563
Wednesday,2760088
Tuesday,2809949
Friday,2891679
Thursday,2913951


It looks like there are fewer violations on weekends, which makes sense! During the week, more people are driving in New York City.

<a name='s1-3.4'></a>
### Let's time it! ###
Loading and processing this data took a little time. Let's measure how long these pipelines take in pandas:

In [ ]:
%%time

df = pd.read_parquet(
    "nyc_parking_violations_2022.parquet",
    columns=["Registration State", "Violation Description", "Vehicle Body Type", "Issue Date", "Summons Number"]
)

(df[["Registration State", "Violation Description"]]
 .value_counts()
 .groupby("Registration State")
 .head(1)
 .sort_index()
 .reset_index()
)

CPU times: user 6.96 s, sys: 1.4 s, total: 8.37 s
Wall time: 7.55 s


,Registration State,Violation Description,count
0,99,<NA>,17550
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
62,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
63,WA,21-No Parking (street clean),3732
64,WI,14-No Standing,1639
65,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


In [ ]:
%%time

(df
 .groupby(["Vehicle Body Type"])
 .agg({"Summons Number": "count"})
 .rename(columns={"Summons Number": "Count"})
 .sort_values(["Count"], ascending=False)
)

CPU times: user 1.01 s, sys: 221 ms, total: 1.23 s
Wall time: 1.35 s


,Count
Vehicle Body Type,
SUBN,6449007
4DSD,4402991
VAN,1317899
DELV,436430
PICK,429798
...,...
OLLL,1
OLLE,1
OLHE,1


In [ ]:
%%time

weekday_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["Issue Date"] = df["Issue Date"].astype("datetime64[ms]")
df["issue_weekday"] = df["Issue Date"].dt.weekday.map(weekday_names)

df.groupby(["issue_weekday"])["Summons Number"].count().sort_values()

CPU times: user 11.2 s, sys: 428 ms, total: 11.6 s
Wall time: 11.7 s


,Summons Number
issue_weekday,
Sunday,462992
Saturday,1108385
Monday,2488563
Wednesday,2760088
Tuesday,2809949
Friday,2891679
Thursday,2913951
